# Homework 7: Scraping part 2

## Part Two

Scrape the content of https://www.lemonde.fr/Links to an external site. and save it as a CSV.

We want: titles, subhead, article URL, whether it's premium or not, byline, article type, image URL.

- Bonus, if you want to get fancy:

Make the CSV file auto-updating. Use this tutorial (videoLinks to an external site., textLinks to an external site.) but just ignore the visualization/datawrapper aspect


In [1]:
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import requests
url="https://www.lemonde.fr/en/"

response = requests.get(url)
doc = BeautifulSoup(response.text)

In [3]:
items = doc.find_all(class_="article")

In [4]:
len(items)

53

In [5]:
items[0]

<a class="article article--nav" data-suggestion="" href="https://www.lemonde.fr/en/international/article/2025/06/13/israel-launches-massive-lightning-attack-on-iran_6742310_4.html"> <div class="article__media-container"> <picture class="article__media"> <img alt="A damaged building after Israeli strikes in Tehran, Iran, on June 13, 2025." class="teaser__media teaser__media--nav js-media-nav" data-lazy="https://img.lemde.fr/2025/06/13/0/0/8640/5760/180/0/95/0/e15292d_upload-1-hhjo8hlmhqck-421634.jpg" data-lazy-retina="https://img.lemde.fr/2025/06/13/0/0/8640/5760/360/0/95/0/e15292d_upload-1-hhjo8hlmhqck-421634.jpg 2x" height="120" width="180"/> <noscript><img alt="A damaged building after Israeli strikes in Tehran, Iran, on June 13, 2025." height="120" src="https://img.lemde.fr/2025/06/13/0/0/8640/5760/180/0/95/0/e15292d_upload-1-hhjo8hlmhqck-421634.jpg" srcset="https://img.lemde.fr/2025/06/13/0/0/8640/5760/360/0/95/0/e15292d_upload-1-hhjo8hlmhqck-421634.jpg 2x" width="180"/></noscript>

In [6]:
rows = []

for item in items:
    row = {}

    #Title
    try:
        row["title"] = item.find(class_="article__title-label").text
    except:
        row["title"] = item.find(class_="article__title").text

    #Subhead
    try:
        row["subhead"] = item.find(class_="article__desc").text
    except:
        row["subhead"] = 0

    #URL
    try:
        row["url"] = item.find("a")["href"]
    except:
        row["url"] = item.get("href")

    #Suscription
    try:
        row["suscription"] = item.find(class_="sr-only").text
    except:
        row["suscription"] = "Not premium"

    #Article type
    try:
        row["article_type"] = item.find(class_="article__type")
    except:
        pass

    rows.append(row)

len(rows)

53

In [7]:
rows[0]

{'title': 'Israel launches massive lightning attack on Iran',
 'subhead': 0,
 'url': 'https://www.lemonde.fr/en/international/article/2025/06/13/israel-launches-massive-lightning-attack-on-iran_6742310_4.html',
 'suscription': 'Subscribers only',
 'article_type': None}

In [8]:
#Now for each news article

url = "https://www.lemonde.fr/en/international/article/2025/06/11/aragon-and-catalonia-battle-over-the-sistine-chapel-of-romanesque-art_6742260_4.html"

response = requests.get(url)
doc = BeautifulSoup(response.text)

In [9]:
print(doc.find(class_="meta__author").text)

  By Sandrine Morel (Madrid (Spain) special correspondent) 


In [10]:
articles_info = []

for row in rows:
    article_info = {}
    article_url = row["url"]

    response = requests.get(article_url)
    doc = BeautifulSoup(response.text)

    #Byline
    try:
        article_info["byline"] = doc.find(class_="meta__authors").text
    except:
        try:
            article_info["byline"] = doc.find(class_="meta__author").text
        except:
            try:
                article_info["byline"] = doc.find(class_="article__author-identity").text
            except:
                article_info["byline"] = "No byline"

    #Image
    article_info["img_url"] = doc.find("img").get("data-lazy-retina")

    articles_info.append(article_info)

len(articles_info)

53

In [11]:
articles_info[0]

{'byline': ' By\xa0Samuel Forey\xa0(Jerusalem, correspondent) and Ghazal Golshiri ',
 'img_url': 'https://img.lemde.fr/2025/06/13/0/0/8640/5760/360/0/95/0/e15292d_upload-1-hhjo8hlmhqck-421634.jpg 2x'}

In [12]:
#Now nesting one inside another to get a new df
rows = []

for item in items:
    row = {}

    #Title
    try:
        row["title"] = item.find(class_="article__title-label").text
    except:
        row["title"] = item.find(class_="article__title").text

    #Subhead
    try:
        row["subhead"] = item.find(class_="article__desc").text
    except:
        row["subhead"] = 0

    #URL
    try:
        row["url"] = item.find("a")["href"]
    except:
        row["url"] = item.get("href")

    #Suscription
    try:
        row["suscription"] = item.find(class_="sr-only").text
    except:
        row["suscription"] = "Not premium"

    #Article type
    try:
        row["article_type"] = item.find(class_="article__type").text
    except:
        pass

    #Info for each article
    article_url = row["url"]
    response = requests.get(article_url)
    doc = BeautifulSoup(response.text)

    #Byline
    try:
        row["byline"] = doc.find(class_="meta__authors").text
    except:
        try:
            row["byline"] = doc.find(class_="meta__author").text
        except:
            try:
                row["byline"] = doc.find(class_="article__author-identity").text
            except:
                row["byline"] = "No byline"

    #Image
    row["img_url"] = doc.find("img").get("data-lazy-retina")

    rows.append(row)

len(rows)

53

In [13]:
rows[-1]

{'title': "Los Angeles: Donald Trump's calculated escalation",
 'subhead': 0,
 'url': 'https://www.lemonde.fr/en/opinion/article/2025/06/10/los-angeles-donald-trump-s-calculated-escalation_6742196_23.html',
 'suscription': 'Not premium',
 'article_type': 'Editorial',
 'byline': 'Le Monde',
 'img_url': 'https://img.lemde.fr/2025/06/13/0/0/8640/5760/360/0/95/0/e15292d_upload-1-hhjo8hlmhqck-421634.jpg 2x'}

In [14]:
df = pd.json_normalize(rows)

In [15]:
df.head()

,title,subhead,url,suscription,byline,img_url,article_type
0,Israel launches massive lightning attack on Iran,0,https://www.lemonde.fr/en/international/articl...,Subscribers only,"By Samuel Forey (Jerusalem, correspondent) an...",https://img.lemde.fr/2025/06/13/0/0/8640/5760/...,NaN
1,"As Israel strikes Iran, US pointedly steps bac...",0,https://www.lemonde.fr/en/international/articl...,Subscribers only,By Piotr Smolar (Washington (United States) ...,https://img.lemde.fr/2025/06/13/0/0/8640/5760/...,NaN
2,Federal judge rules Trump's deployment of Nati...,0,https://www.lemonde.fr/en/international/articl...,Subscribers only,"By Corine Lesnes (Los Angeles, special corre...",https://img.lemde.fr/2025/06/13/0/0/8640/5760/...,NaN
3,War in Ukraine: Russia steps up its offensive ...,0,https://www.lemonde.fr/en/international/articl...,Subscribers only,By Emmanuel Grynszpan,https://img.lemde.fr/2025/06/13/0/0/8640/5760/...,NaN
4,Criticism is mounting in the Netherlands again...,0,https://www.lemonde.fr/en/international/articl...,Subscribers only,By Jean-Pierre Stroobants (The Hague (Nether...,https://img.lemde.fr/2025/06/13/0/0/8640/5760/...,NaN


In [17]:
df.to_csv("le_monde.csv", index=False)